# Defining your own type


Types can be thought as "boxes" containing data. Each "box" is called  **``field``** and can contain any  **``Type``** inside, each field is identified by a name filed. In order to see the different fields of a type you can use the function **``fieldnames``**.

- Each field can have a type, the symbol **`::`** is used to assign a type to a fieldname.
    - For example **`x::Float64`** states that `x` has type ``Float64``.
- Each field can have a different type

In [377]:
workspace()
type Point
    x::Float64
    y::Float64
    z::Float64
end
p = Point(10,10,2)

Point(10.0,10.0,2.0)

In [378]:
fieldnames(p)

3-element Array{Symbol,1}:
 :x
 :y
 :z

In [379]:
p.x = 23

23

In [380]:
p

Point(23.0,10.0,2.0)

## Inmutable Types

As the name suggests, you cannot mutate the fieldnames of **``immutable``** types.


**An immutable object might contain mutable objects**, such as arrays, as fields. Those contained objects will remain mutable; only the fields of the immutable object itself cannot be changed to point to different objects.

In [361]:
workspace()
immutable Point
    x::Float64
    y::Float64
    z::Float64
end
p = Point(10,10,2)

Point(10.0,10.0,2.0)

In [362]:
p.x = 23

LoadError: type Point is immutable

In [363]:
workspace()
immutable VectorPoint
    x::Array{Float64}
    y::Array{Float64}
    z::Array{Float64}
end
p = VectorPoint([10,2],[10,1],[10,10])

VectorPoint([10.0,2.0],[10.0,1.0],[10.0,10.0])

In [367]:
p.x = [2.,3.]

LoadError: type VectorPoint is immutable

In [394]:
immutable Family
    members::Array{String}
    name::String
end

In [401]:
simpsons = Family(["Lisa","Hommer","Bart","Marge"],"Simpson")

Family(String["Lisa","Hommer","Bart","Marge"],"Simpson")

In [402]:
push!(simpsons.members, "maggie")

5-element Array{String,1}:
 "Lisa"  
 "Hommer"
 "Bart"  
 "Marge" 
 "maggie"

In [371]:
simpsons

Family(String["Lisa","Hommer","Bart","Marge","maggie"],"Simpson")

In [384]:
immutable Model
    name::String
    Θ::Array
end

In [417]:
perceptron = Model("Percepton", zeros(10))

Model("Percepton",[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0])

In [418]:
perceptron.Θ[1] = 12

12

In [421]:
perceptron.Θ'

1×10 Array{Float64,2}:
 13.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0

In [422]:
perceptron.Θ = perceptron.Θ +1

LoadError: type Model is immutable

In [420]:
perceptron.Θ .= perceptron.Θ +1

10-element Array{Float64,1}:
 13.0
  1.0
  1.0
  1.0
  1.0
  1.0
  1.0
  1.0
  1.0
  1.0

## Parametric types

To make the code more general we can use ** parametric types** which define a family of types by a parameter. 

- Parametric types can depend on multiple parameters `Point{T1,T2}`.


In [310]:
workspace()
type Point{T}
    x::T
    y::T
    z::T
end

In [311]:
p = Point(10,10,2)

Point{Int64}(10,10,2)

In [312]:
p = Point(10.,10.,2.)

Point{Float64}(10.0,10.0,2.0)

#### This code
``` Julia
type Point{T}
    x::T
    y::T
    z::T
end
```

#### Already defines all this


``` Julia
type Point{T<:Real}
  x::T
  y::T

  Point(x,y,z) = new(x,y,z)
end

Point{T<:Real}(x::T, y::T, z::T) = Point{T}(x,y,z)
```

In [208]:
workspace()
type Point{T1,T2}
    x::T1
    y::T1
    z::T2
end

In [210]:
p = Point(10.,10.,2)

Point{Float64,Int64}(10.0,10.0,2)

## Set restrictions on custom types


Sometimes we don't want to have types that are too general and can be missused in unnexpected ways. In order to avoid this we can put restrictions. For example, our `Point` type can be created to be used only with numbers inside its fieldnames, nevertheless, our previous definition

``` Julia
type Point{T}
    x::T
    y::T
    z::T
end
```

can accept any `Type` inside `x`, `y` and `z` as long as it's the same type for all fieldnames. 

``` Julia
Point("This", "is", "unnexpected")
```
```
Point{String}("This","is","unnexpected")
```

#### Using the subtype operator <:

The operator **`<:`** allows us to verify if a `Type` is subtype of an `AbstractType`.


In [260]:
Int64 <: Real

true

In [261]:
workspace()
type Point{T<:Real}
    x::T
    y::T
    z::T
end

In [262]:
Point("This", "is", "unnexpected")

LoadError: MethodError: no method matching Point{T<:Real}(::String, ::String, ::String)[0m
Closest candidates are:
  Point{T<:Real}{T}(::Any) at sysimg.jl:53[0m

In [264]:
p = Point(10,10,2)

Point{Int64}(10,10,2)

## Set default values for a custom type

Constructors can be used to generate ways to construct custom types. This could be usefull, for example, in order to set default values to your custom types.

In [589]:
workspace()
type Point{T}
    x::T
    y::T
    z::T
end

# Make a constructor that sets some default values 
Point(x,y) = Point(x,y,0)

# Make a constructor that sets all values as default
Point() = Point(0,0,0)

Point{T}

In [590]:
Point(10,10)

Point{Int64}(10,10,0)

In [591]:
Point()

Point{Int64}(0,0,0)

## Set restrictions for custom types, inner constructors

If we want to enforce properties of a type in its construction we can use inner constructors with restrictions

In [623]:
type OrderedPair
  x::Real
  y::Real
   
  OrderedPair(x,y) = x > y ? error("out of order") : new(x,y)
end

In [628]:
OrderedPair(1,10)

OrderedPair(1,10)

In [643]:
OrderedPair(10,1)

LoadError: UndefVarError: OrderedPair not defined

#### Why is this not working?

In [676]:
workspace()
type Point{T}
    x::T
    y::T
    z::T
    function Point(x,y,z)
        if x < 0 error("x should be positive") end
        if y < 0 error("y should be positive") end
        if z < 0 error("z should be positive") end
        return new(x,y,z)
    end 
end

In [677]:
Point{T}(x::T,y::T,z::T) =Point(x,y,z)

Point{T}

In [679]:
Point(3,3,4)

LoadError: StackOverflowError:

In [660]:
Point(-23,3,4)

LoadError: MethodError: no method matching Point{T}(::Int64, ::Int64, ::Int64)[0m
Closest candidates are:
  Point{T}{T}(::Any) at sysimg.jl:53[0m

In [663]:
workspace()
type Point
    x
    y
    z
    
    function Point(x,y,z)
        if x < 0 error("x should be positive") end
        if y < 0 error("y should be positive") end
        if z < 0 error("z should be positive") end
        return new(x,y,z)
    end
    
end

In [664]:
Point(3,3,4)

Point(3,3,4)

In [665]:
Point(-23,3,4)

LoadError: x should be positive

### Creating inner constructors

We can add a function inside a type used to instanciate the type.

This function can be used for example in order to..

- Put restrictions on the type, such as..
     - not allow negative values 
     - Not allow 'rare' characters

In [550]:
workspace()
type Giant{T}
    weight::T
    height::T
    intelligence::String
    
    function Giant(w, h, i)
        if h<2
            throw(ArgumentError("Giants are taller\n"))
        end
        return new(w,h,i)
    end
end

In [551]:
methodswith(Giant)

0-element Array{Method,1}

In [552]:
Giant{T}(weight::T, height::T, intelligence::String) = Giant{T}(weight, height, intelligence)

Giant{T}

In [553]:
Giant(3.,4.,"low")

Giant{Float64}(3.0,4.0,"low")

In [554]:
giant = Giant(3.,1.,"medium")

LoadError: ArgumentError: Giants are taller


In [ ]:
type Family
    name::AbstractString
    members::Array{AbstractString, 1}
    extended::Bool
    # constructor that takes one argument and generates a default
    # for the other two values
    Family(name::AbstractString) = new(name, AbstractString[], false)
    # constructor that takes two arguements and infers the third
    Family(name::AbstractString, members) = new(name, members, length(members) > 3)
end

fam1 = Family("blogs")
println(fam1)

Notice that if a constructor is defined inside a type we must explicitly define a way to instanciate new elements for every possible type T.

Let us do it

In [465]:
Giant{T}(weight::T, height::T, intelligence::String) = Giant{T}(weight, height, intelligence)

#Giant{T}(weight::T, height::T) = Giant{T}(weight, height ; intelligence="low")

Giant{T}

In [466]:
methodswith(Giant)

0-element Array{Method,1}

In [467]:
giant = Giant(3., 4., "low")

Giant{Float64}(3.0,4.0,"low")

In [468]:
giant.height, giant.weight, giant.intelligence

(4.0,3.0,"low")

Notice that, sice we have written a restriction on the height of the giant
we cannot instanciate giants with less than 2 meter height

In [469]:
giant = Giant(3.,4.,"medium")

Giant{Float64}(3.0,4.0,"medium")

In [470]:
giant.height, giant.weight, giant.intelligence

(4.0,3.0,"medium")

In [471]:
giant = Giant(3.,1.,"medium")

LoadError: ArgumentError: Giants are taller


### Assigning  a default value to a type 

By default we might think that Giants are quite stupid and therefore we might be interested into assigning "stupid" intelligence as default.

Notice that default values in Julia are defined after ```;```

This can be specially handly in order to provide default values for types, for example when there could be 'standard values' for some of the fileds inside a type and we do not want to bother the user selecting field values. 

Using this trick we can allow the user to set values to all the fields the user wants to but at the same time, allow the user **not to** define all possible fields .

In [ ]:
workspace()

In [ ]:
type Giant{T}
    weight::T
    height::T
    intelligence::String
    
    function Giant(w, h ; intelligence="stupid")
        if h<2
            throw(ArgumentError("Giants are taller\n"))
        end
        return new(w, h ,  intelligence)
    end
    
end

## IMPORTANT TO ADD CONSTRUCTOR FOR EVERY TYPE T !!!!
Giant{T}(weight::T, height::T) = Giant{T}(weight, height)


In [ ]:
giant = Giant(3.,5.)

In [ ]:
giant.weight, giant.height, giant.intelligence

In [ ]:
giant.intelligence = "smart"

In [ ]:
giant.weight, giant.height, giant.intelligence

Notice that we cannot pass now a value for the field intelligence

In [ ]:
rare_giant = Giant(5.,4.,"smart")

Notice that we cannot use this definition since we need to specify how to create a Giant  when we pass the intelligence field.

### Allowing types to have fields with defeault values

https://groups.google.com/forum/#!topic/julia-users/9jM7GIoh7YY

In [ ]:
workspace()

type Giant{T}
    weight::T
    height::T
    intelligence::String
    
    function Giant(w, h ; intelligence="stupid")
        if h<2
            throw(ArgumentError("Giants are taller\n"))
        end
        return new(w, h, intelligence)
    end
    
    function Giant(w, h, intelligence)
        if h<2
            throw(ArgumentError("Giants are taller\n"))
        end
        return new(w, h, intelligence)
    end
end

## IMPORTANT TO ADD CONSTRUCTOR FOR EVERY TYPE T !!!!
Giant{T}(weight::T, height::T) = Giant{T}(weight, height)
Giant{T}(weight::T, height::T, intelligence::String) = Giant{T}(weight, height, intelligence)

In [ ]:
rare_giant = Giant(5., 4., "smart")

In [ ]:
giant = Giant(3., 5.)

#### Type man


The following example is the type man that contains 3 fields, **```heigh```**,**``` weight```** and **```name```**.

In [ ]:
type man{T}
    height::T
    weight::T
    name::String
end

A type can be instanciated by assigning values to its fields, the fields of a type can be checked with the **```fieldnames```** function

In [ ]:
david = man(183, 80, "David")

In [ ]:
fieldnames(david)

Notice that **you cannot specify a particular value on a field**.

In [ ]:
type girl{T}
    height::T
    weight::T
    name="Julia"
end

Nevertheless you can define a type and a (so called) inner constructor which might specify a particular value of the type.

In [ ]:
type girl2{T}
    height::T
    weight::T
    name::String
end

In [ ]:
girl2(23, 123, "julia")

In [ ]:
typeof(girl2)

In [ ]:
methodswith(girl2)

### Inner constructor

In [ ]:
type girl4{T}
    height::T
    weight::T
    name::String
    
    function girl4(height, weight, name)
        new(23,12,"lala")
    end
end

In [ ]:
girl4(23,123,"julia")

# Making custom print for a defined type

Once defined a type we can import ```Base.show``` and define a ```show``` method for the defined type


In [ ]:
import Base.show

In [ ]:
type Point
       x::Int
       y::Int
end

In [ ]:
Base.show(io::IO, p::Point) = print(io, "Point: x=$(p.x), y=$(p.y)")

In [ ]:
#Base.print(io::IO, p::Point) = string(p)

In [ ]:
Point(4,5)

In [ ]:
print(Point(4,5))

In [ ]:
[Point(2,5), Point(1,1), Point(5,6)]

### Another example

In [ ]:
require("Enum")

using Calendar
   type LogMessage
     stamped::CalendarTime
     msgstat::MsgStatus
     message::String
end
   import Base.show
   show(m::LogMessage) =
   print("$(m.stamped): $(m.msgstat) >> $(m.message)")
   msg = LogMessage(Calendar.now(), WARN, "Be very afraid")
   show(msg)

### The importance of declaring code inside functions

In [ ]:
@time begin
pos = 0
num_steps = 10^4
numwalkers = 10^4
final_square_positions = Int[]

for i in 1:numwalkers
    for j in 1:num_steps
        pos += ifelse( rand() < 0.5, -1, +1)
    end
        push!(final_square_positions, pos^2)
end
    
end

In [ ]:
function fool_function()
    pos = 0
    num_steps = 10^4
    numwalkers = 10^4
    final_square_positions = Int[]

    for i in 1:numwalkers
        for j in 1:num_steps
            pos += ifelse( rand() < 0.5, -1, +1)
        end
            push!(final_square_positions, pos^2)
    end
end

In [ ]:
@time fool_function()